In [1]:
import pandas as pd
import numpy as np
import re

In [16]:
def is_IP(token):
  index = 0
  indexs = []
  for word in token.split(" "):
      # print(word)
      ip = re.findall(r"\b\d{1,3}\.\d{1,3}\.\d{1,3}\.\d{1,3}\b", word)
     
      if len(ip) != 0:
          index_start_ip = word.find(ip[0]) + index
          
          index_end_ip = index_start_ip + len(ip[0])
          
          indexs.append((index_start_ip, index_end_ip))
      index += len(word) + 1
  return indexs

In [17]:
is_IP("192.0.2.1")

['.1']


[(7, 9)]

# Convert .Coll sang DataFrame

In [29]:
def read_conll(in_file, lowercase=False, max_example=None):
    f_name = in_file.split("/")[-1].replace(".conll", "")
    examples = []
    
    with open(in_file) as f:
        n_sents, word, label1, label2, label3= [], [], [], [], []
        n_sent = 0
        for line in f.readlines():
            
            sp = line.strip().split('\t')
            if sp[0] != '_':
                if len(sp) >= 4:
                    part = sp[3].split('-', 1)
                    if len(part) > 1:
                        postion = part[0]
                        label_name = part[1]
                    else:
                        postion = 'default'
                        label_name = part[0]

                    if postion == 'I' and label1[-1] == label_name:
                        if label_name not in ['URL', 'EMAIL', 'PHONENUMBER']:
                            word[-1] = word[-1] + ' ' + sp[0]
                        else:
                            word[-1] = word[-1] + sp[0]

                    else:
                        word.append(sp[0])
                        label1.append(label_name)
                        label2.append('?')
                        label3.append('?')
                        n_sents.append(n_sent)
                        
                if len(sp) == 5:
                    label2[-1] = sp[4]
                if len(sp) == 6:
                    label3[-1] = sp[5]
                n_sent += 1  
     
            # save sentence to examples
            if sp[0] == '':
                if word[-1] != '.':
                    word.append('.')
                    label1.append('O')
                    label2.append('END')
                    label3.append('END')
                    n_sents.append(n_sent)

                examples.append({'f_name':f_name,'n_sent':n_sents, 'word': word, 'label1': label1, 'label2': label2, 'label3': label3})
                n_sent = 0
           
            
    return examples 

In [11]:
'I-PHONENUMBER-MANH'.split('-', 1)

['I', 'PHONENUMBER-MANH']

In [35]:
dff = read_conll('/Users/phamvanmanh/Desktop/machine/final_vlsp/converted/train-converted/00_add_0269.conll')
df1 = pd.DataFrame(dff[0])
df1[df1['label1'] == 'PHONENUMBER']

,f_name,n_sent,word,label1,label2,label3
399,00_add_0269,58,0987673608,PHONENUMBER,O,?


In [6]:
def to_zip(df, parent_path, fold_name):

    compression_opts = dict(method='zip',
                                archive_name=fold_name + '.csv')  
    df.to_csv( parent_path + fold_name + '.zip', index=False,
                compression=compression_opts) 
    return print(fold_name + " saved done!")

In [7]:
import os

def convert_Dataframe(in_folder, out_folder):
    
    sub_folders = [name for name in os.listdir(in_folder) if os.path.isdir(os.path.join(in_folder, name))]
    print(sub_folders)
    for n_folder in sub_folders:
        df = pd.DataFrame(columns = ['f_name','n_sent', 'word', 'label1', 'label2', 'label3'])
        path = in_folder + n_folder
        sub_files = os.listdir(path)
    
  
        for s_file in sub_files:
            try:
                f_path = path + '/' + s_file
                data = read_conll(f_path)
                df1 = pd.DataFrame(data[0])
                df = df.append(df1)
            except  Exception as e: print('error read_conll: {}, {}'.format(e, f_path))

        try:
            to_zip(df, out_folder, n_folder)
        except  Exception as e: print('error to_zip: {}'.format(e))
        


    return 'done!'

        

x

In [8]:
out_folder = '/Users/phamvanmanh/Desktop/machine/final_vlsp/converted_to_dataframe/'
in_folder = '/Users/phamvanmanh/Desktop/machine/final_vlsp/converted/'

In [36]:
convert_Dataframe(in_folder, out_folder)

['test-converted', 'dev-converted', 'train-converted']
test-converted saved done!
dev-converted saved done!
train-converted saved done!


'done!'